In [1]:
from lib.KDTreeEncoding import *

import xgboost as xgb
from lib.XGBHelper import *
from lib.XGBoost_params import *
from lib.score_analysis import *

from lib.logger import logger

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from numpy import load
from glob import glob
import pandas as pd
import pickle as pkl

poverty_dir = '/datasets/cs255-sp22-a00-public/poverty/'
image_dir=poverty_dir+'anon_images/'

In [2]:
import pickle as pkl
pkl_file='data/DumpNoSplit.pk'
D=pkl.load(open(pkl_file,'rb'))

for k in D:
    globals()[k]=D[k]
scaling_mean=mean
scaling_std=std

bst_list=[x['bst'] for x in styled_logs[1]['log']]

In [3]:
folds=[{'in':'country_test_reduct.csv','out':'results_country.csv'},
      {'in':'random_test_reduct.csv','out':'results.csv'}]

for fold_i in range(len(folds)):
    fold=folds[fold_i]

    test_csv=f'../public_tables/{fold["in"]}'
    test=pd.read_csv(test_csv,index_col=0)
#     test = test[:100]
    out=pd.DataFrame()
    out['filename'] = test['filename']
    out['urban']=test['urban']
    out['country']=test['country']
    out['pred_wo_abstention']=0
    out.set_index('filename', inplace=True)
    
    Enc_data=encoded_dataset(image_dir,out,tree,label_col='pred_wo_abstention')


    data=to_DMatrix(Enc_data.data)
    Preds=zeros([Enc_data.data.shape[0],len(bst_list)])
    for i in range(len(bst_list)):
        Preds[:,i]=bst_list[i].predict(data,output_margin=True,ntree_limit=bst_list[i].best_ntree_limit)
#     Preds=(Preds-scaling_mean)/scaling_std # apply overall score scaling
    _mean=np.mean(Preds,axis=1)
    print('mean',_mean[out['urban']==True])
    _std=np.std(Preds,axis=1)
    pred_wo_abstention = np.zeros((_mean.shape))
    print(pred_wo_abstention)
    pred_wo_abstention[out['urban']==True]=(2*(_mean[out['urban']==True]>1.3))-1
    print(pred_wo_abstention)
    pred_wo_abstention[out['urban']==False]=(2*(_mean[out['urban']==False]>(-0.5)))-1
    print(pred_wo_abstention)
    pred_with_abstention=copy(pred_wo_abstention)
    pred_with_abstention[_std>abs(_mean)]=0
    print(pred_with_abstention)

    out['pred_with_abstention'] = pred_with_abstention
    out['pred_wo_abstention'] = pred_wo_abstention
    out['pred_with_abstention'] = out['pred_with_abstention'].astype(int)
    out['pred_wo_abstention'] = out['pred_wo_abstention'].astype(int)
    outFile=f'data/{fold["out"]}'
    out.to_csv(outFile)
    print('\n\n'+'-'*60)
    print(outFile)




mean [0.37125853 0.25611726 0.4765992  ... 0.37080099 0.46502017 0.35390972]
[0. 0. 0. ... 0. 0. 0.]
[ 0. -1. -1. ... -1. -1.  0.]
[ 1. -1. -1. ... -1. -1.  1.]
[ 1. -1. -1. ... -1. -1.  0.]


------------------------------------------------------------
data/results_country.csv
mean [1.18766821 0.81903897 0.55335873 ... 0.23423295 0.32743389 0.26219404]
[0. 0. 0. ... 0. 0. 0.]
[ 0. -1. -1. ... -1. -1. -1.]
[ 1. -1. -1. ... -1. -1. -1.]
[ 0. -1. -1. ...  0. -1. -1.]


------------------------------------------------------------
data/results.csv


In [4]:
out.dtypes

urban                    bool
country                 int64
pred_wo_abstention      int64
pred_with_abstention    int64
dtype: object

In [9]:
_mean[out['urban']==False].min()

-0.799342131614685

In [8]:
_mean[out['urban']==True].mean()

0.3934245919279948